In [13]:
import sys
import os
# Thêm thư mục gốc của project vào sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [14]:
import json
import os
import sys
# Thêm thư mục gốc của project vào sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.api_client import RAGAPIClient
# Initialize API client
api_client = RAGAPIClient(base_url="http://localhost:8000/api", endpoint='query')

# Check if API is available
if not api_client.check_health():
    print("⚠️ RAG API is not available at http://localhost:8000")
    print("Please start the RAG service first!")
else:
    print("✅ RAG API is available")
def load_questions(file_path):
    """Load questions from a JSON file"""
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)




2025-12-01 17:07:38.976 | INFO     | utils.api_client:__init__:24 - Initialized RAG API Client: http://localhost:8000/api


⚠️ RAG API is not available at http://localhost:8000
Please start the RAG service first!


In [ ]:
def generate_rag_answers(questions, api_client):
    """Generate RAG answers for a list of questions"""
    rag_responses = []
    
    for i, tc in enumerate(questions):
        question = tc.get('question', '')
        print(f"[{i}] Generating answer for: {question[:50]}...")
        
        try:
            response = api_client.ask_question(question)
            contexts = [f'{source.get('text_preview')}'for source in response.get('sources', [])]
            
            rag_responses.append({
                'question_id': tc.get('id', i),
                'ground_truth': tc.get('ground_truth', ''),
                'question': question,
                'answer': response.get('answer', ''),
                'contexts': contexts
                'processing_time': response.get('metadata', {}).get('total_time_ms', None)
            })
            
            print(f"  ✅ Got answer ({len(response.get('answer', ''))} chars)")
        
        except Exception as e:
            print(f"  ❌ Error: {e}")
            rag_responses.append({
                'question_id': tc.get('id', i),
                'question': question,
                'answer': '',
                'contexts': [],
                'error': str(e)
            })
    
    print(f"\n✅ Generated {len([r for r in rag_responses if r.get('answer')])} answers")
    return rag_responses

def save_results(rag_responses, output_path):
    """Save RAG responses to a JSON file"""
    with open(output_path, "w", encoding="utf-8-sig") as f:
        json.dump(rag_responses, f, ensure_ascii=False, indent=2)
    print(f"✅ Saved {len(rag_responses)} responses to {output_path}")

def process_folder(input_folder, output_folder, api_client):
    """Process all JSON files in the input folder"""
    os.makedirs(output_folder, exist_ok=True)

    input_path = os.path.join(input_folder, "questions.json")
    output_path = os.path.join(output_folder, filename.replace("questions", "results"))



    
    for filename in os.listdir(input_folder):
        if filename.endswith(".json"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename.replace("questions", "results"))
            
            print(f"\n📂 Processing file: {input_path}")
            questions = load_questions(input_path)
            responses = generate_rag_answers(questions, api_client)
            save_results(responses, output_path)

In [ ]:
process_folder("../data/testcase/factual_testcase", "../data/testcase/factual_testcase_results", api_client)

2025-12-01 17:08:22.913 | INFO     | utils.api_client:ask_question:43 - Asking question: Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu...



📂 Processing file: ../data/testcase/factual_testcase/questions.json
[0] Generating answer for: Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu...
{'query': 'Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu tín chỉ để tốt nghiệp?'}


2025-12-01 17:08:30.226 | INFO     | utils.api_client:ask_question:52 - Response data: {'answer': 'Để được công nhận tốt nghiệp và cấp bằng, sinh viên ngành Trí Tuệ Nhân Tạo phải tích lũy tối thiểu 128 tín chỉ.', 'sources': [{'section_id': '29745874eefd', 'title': '7\\. ĐIỀU KIỆN TỐT NGHIỆP', 'hierarchy_path': 'Ngành Trí Tuệ Nhân Tạo > 7\\. ĐIỀU KIỆN TỐT NGHIỆP', 'text_preview': '[Ngành Trí Tuệ Nhân Tạo > 7\\. ĐIỀU KIỆN TỐT NGHIỆP]\n\nĐể được công nhận tốt nghiệp và cấp bằng, sinh viên phải:\n1\\. Hoàn thành các môn học bắt buộc của chương trình đào tạo.\n2\\. Tích lũy tối thiểu\xa0**128**\xa0tín chỉ được phân bố theo các khối kiến thức sau:\n\\- Kiến thức giáo dục đại cương: tối thiểu 45 tín chỉ;\n\\- Kiến thức cơ sở ngành: tối thiểu 57 tín chỉ;\n\\- Kiến thức tự chọn ngành: tối thiểu 8 tín chỉ;\n\\- Kiến thức tự chọn liên ngành: tối thiểu 8 tín chỉ;\n\\- Kiến thức tốt nghiệp: tối thiểu 10 tín chỉ.\n3\\. Đáp ứng các điều kiện khác theo quy chế đào tạo hiện hành của Trường Đại học Công

  ✅ Got answer (111 chars)
[1] Generating answer for: Thời gian đào tạo ngành Công Nghệ Thông Tin kéo dà...
{'query': 'Thời gian đào tạo ngành Công Nghệ Thông Tin kéo dài bao nhiêu năm?'}


2025-12-01 17:08:35.112 | INFO     | utils.api_client:ask_question:52 - Response data: {'answer': 'Thời gian đào tạo ngành Công Nghệ Thông Tin kéo dài 4 năm (8 học kỳ chính).', 'sources': [{'section_id': 'c23779799702', 'title': 'Hình thức và thời gian đào tạo', 'hierarchy_path': 'Ngành Công Nghệ Thông Tin > Hình thức và thời gian đào tạo', 'text_preview': '[Ngành Công Nghệ Thông Tin > Hình thức và thời gian đào tạo]\n\n\\- Hình thức đào tạo: Chính quy tập trung.\n\n\\- Số tín chỉ đào tạo: Tối thiểu 125 tín chỉ (bao gồm cả ngoại ngữ).\n\n\\- Thời gian đào tạo: 04 năm (8 học kỳ chính).', 'score': 0.8902836}, {'section_id': '54eddb92ca05', 'title': 'Hình thức và thời gian đào tạo', 'hierarchy_path': 'Ngành Kỹ Thuật Máy Tính > Hình thức và thời gian đào tạo', 'text_preview': '[Ngành Kỹ Thuật Máy Tính > Hình thức và thời gian đào tạo]\n\n- Hình thức đào tạo: Chính quy tập trung.\n\n- Số tín chỉ đào tạo: 128 tín chỉ.\n\n- Thời gian đào tạo: 4 năm (8 học kỳ)', 'score': 0.85994816}, {'section

...

2025-12-01 17:10:07.886 | INFO     | utils.api_client:ask_question:52 - Response data: {'answer': 'Dựa trên thông tin được cung cấp, các môn học tự chọn liên ngành thường được khuyến nghị cho sinh viên AI để phát triển kỹ năng khoa học dữ liệu là:\n\n*   Học máy thống kê (DS102)\n*   Thu thập và tiền xử lý dữ liệu (DS103)\n*   Phân tích dữ liệu lớn (DS200)', 'sources': [{'section_id': 'e3075fe142eb', 'title': 'Khối kiến thức giáo dục chuyên nghiệp', 'hierarchy_path': 'Ngành Trí Tuệ Nhân Tạo > Tỷ lệ các khối kiến thức > Khối kiến thức giáo dục chuyên nghiệp', 'text_preview': '[Ngành Trí Tuệ Nhân Tạo > Tỷ lệ các khối kiến thức > Khối kiến thức giáo dục chuyên nghiệp]\n\nKhối kiến thức giáo dục chuyên nghiệp gồm tối thiểu **73 tín chỉ**, được tổ chức thành ba nhóm chính, cộng với khối kiến thức tốt nghiệp **10 tín chỉ**, giúp sinh viên có nền tảng vững chắc, chuyên sâu và kỹ năng thực hành cần thiết để hoàn thành chương trình đào tạo:\n\n- **Khối cơ sở ngành 

  ✅ Got answer (255 chars)
[17] Generating answer for: Ngành Trí Tuệ Nhân Tạo có đòi hỏi phải học Mật Mã ...
{'query': 'Ngành Trí Tuệ Nhân Tạo có đòi hỏi phải học Mật Mã Học không?'}


2025-12-01 17:10:12.479 | INFO     | utils.api_client:ask_question:52 - Response data: {'answer': 'Tôi không có đủ thông tin để trả lời câu hỏi này.', 'sources': [{'section_id': '29745874eefd', 'title': '7\\. ĐIỀU KIỆN TỐT NGHIỆP', 'hierarchy_path': 'Ngành Trí Tuệ Nhân Tạo > 7\\. ĐIỀU KIỆN TỐT NGHIỆP', 'text_preview': '[Ngành Trí Tuệ Nhân Tạo > 7\\. ĐIỀU KIỆN TỐT NGHIỆP]\n\nĐể được công nhận tốt nghiệp và cấp bằng, sinh viên phải:\n1\\. Hoàn thành các môn học bắt buộc của chương trình đào tạo.\n2\\. Tích lũy tối thiểu\xa0**128**\xa0tín chỉ được phân bố theo các khối kiến thức sau:\n\\- Kiến thức giáo dục đại cương: tối thiểu 45 tín chỉ;\n\\- Kiến thức cơ sở ngành: tối thiểu 57 tín chỉ;\n\\- Kiến thức tự chọn ngành: tối thiểu 8 tín chỉ;\n\\- Kiến thức tự chọn liên ngành: tối thiểu 8 tín chỉ;\n\\- Kiến thức tốt nghiệp: tối thiểu 10 tín chỉ.\n3\\. Đáp ứng các điều kiện khác theo quy chế đào tạo hiện hành của Trường Đại học Công nghệ Thông tin.', 'score': 0.82691526}, {'section_id': '32d5e

  ✅ Got answer (49 chars)

✅ Generated 18 answers
✅ Saved 18 responses to ../data/testcase/factual_testcase_results/results.json

📂 Processing file: ../data/testcase/factual_testcase/results_old.json
[0] Generating answer for: Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu...
{'query': 'Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu tín chỉ để tốt nghiệp?'}


KeyboardInterrupt: 

In [5]:
import utils.convert_results_to_csv as convert_utils

In [6]:
convert_utils.convert_json_to_csv(
    json_file="../data/rag_results.json",
    csv_file="../data/open_rag_eval_results/rag_results.csv"
)

✅ Converted 6 queries to ../data/open_rag_eval_results/rag_results.csv
   Total rows: 30


In [7]:
import json
import csv
file_path = "../data/testcase/factual_testcase/questions.json"

# Đọc file JSON
with open(file_path, "r", encoding="utf-8") as f:
    questions = json.load(f)

with open("questions.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)  # Bao quanh tất cả giá trị bằng ""
    writer.writerow(["query"])

    for row in questions:
        # Lấy nội dung câu hỏi và loại bỏ newline hoặc dấu đặc biệt gây lỗi
        question = row.get("question", "").replace("\n", " ").replace("\r", " ").strip()
        writer.writerow([question])

## RAPTOR

In [8]:
rag_responses = []
# Generate answers for first N test cases
for i, tc in enumerate(questions):
    question_id = tc['question_id']
    question = tc['question']
    print(f"[{question_id}] Generating answer for: {question[:50]}...")
    
    try:
        # Call RAG API
        response = api_client.ask_question(question)
        print(response)
        
        # Store result
        rag_responses.append({
            'question_id':question_id,
            'question': question,
            'answer': response['answer'],
            'sources': response['sources'],
            'contexts': [context.replace("\n", " ") for context in response.get('contexts', [])],
        })
        
        print(f"  ✅ Got answer ({len(response['answer'])} chars)")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")
        rag_responses.append({
            'question_id': tc['id'],
            'question': question,
            'answer': '',
            'contexts': [],
            'sources': [],
            'error': str(e)
        })

print(f"\n✅ Generated {len([r for r in rag_responses if r.get('answer')])} answers")

2025-12-01 17:07:33.514 | INFO     | utils.api_client:ask_question:43 - Asking question: Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu...
2025-12-01 17:07:33.515 | ERROR    | utils.api_client:ask_question:57 - API request failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10a6d6f10>: Failed to establish a new connection: [Errno 61] Connection refused'))
2025-12-01 17:07:33.516 | INFO     | utils.api_client:ask_question:43 - Asking question: Thời gian đào tạo ngành Công Nghệ Thông Tin kéo dà...
2025-12-01 17:07:33.517 | ERROR    | utils.api_client:ask_question:57 - API request failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10a6d67a0>: Failed to establish a new connection: [Errno 61] Connection refused'))
2025-12-01 17:07:33.517 | 

[F001] Generating answer for: Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu...
{'query': 'Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu tín chỉ để tốt nghiệp?'}
{'success': False, 'error': "HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10a6d6f10>: Failed to establish a new connection: [Errno 61] Connection refused'))", 'answer': '', 'sources': []}
  ✅ Got answer (0 chars)
[F002] Generating answer for: Thời gian đào tạo ngành Công Nghệ Thông Tin kéo dà...
{'query': 'Thời gian đào tạo ngành Công Nghệ Thông Tin kéo dài bao nhiêu năm?'}
{'success': False, 'error': "HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10a6d67a0>: Failed to establish a new connection: [Errno 61] Connection refused'))", 'answer': '', 'sources': []}
  ✅ Got answer (

In [9]:
output_path = "../data/rag_results_raptor.json"

# Ghi ra JSON với định dạng đẹp
with open(output_path, "w", encoding="utf-8-sig") as f:
    json.dump(rag_responses, f, ensure_ascii=False, indent=2)

print(f"✅ Saved {len(rag_responses)} responses to {output_path}")

✅ Saved 18 responses to ../data/rag_results_raptor.json


In [10]:
import utils.convert_results_to_csv as convert_utils

In [11]:
convert_utils.convert_json_to_csv_with_contexts(
    json_file="../data/rag_results_raptor.json",
    csv_file="../data/open_rag_eval_results_raptor/rag_results_raptor.csv"
)

✅ Converted 18 queries to ../data/open_rag_eval_results_raptor/rag_results_raptor.csv
   Total rows: 18


In [12]:
import pandas as pd

file_path = "../data/questions.csv"

# Đọc CSV
df = pd.read_csv(file_path)

# Hiển thị toàn bộ bảng
print(df)



                                               query
0  Ngành Kỹ thuật Máy tính có thể làm những công ...
1  Sinh viên mới vào học nên theo thứ tự học nhữn...
2  Nếu em muốn theo hướng Hệ thống nhúng/IoT thì ...
3  Ngành Kỹ thuật Máy tính có những định hướng ch...
4  Trong Ngành Công Nghệ Thông Tin, môn Thực tập ...
5  Chương trình Trí Tuệ Nhân Tạo có những phương ...
